<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/106_%E5%9E%8B%E3%81%AE%E6%A7%8B%E9%80%A0%E3%81%A8%E3%82%A4%E3%83%B3%E3%82%B9%E3%82%BF%E3%83%B3%E3%82%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

型の構造とインスタンス
======================

型とクラス
----------

**クラス**（class）は、オブジェクトの型を定義する仕組みであり、その型が持つデータ（属性）と振る舞い（メソッド）をまとめて表す。クラスを基に生成されたオブジェクトを**インスタンス**（instance）と呼び、インスタンスを生成することを**インスタンス化**（instantiation）という。

Python のクラス機構については、[公式チュートリアル](https://docs.python.org/ja/3/tutorial/classes.html#a-first-look-at-classes) でほぼ十分。

他のプログラミング言語とクラス機構を比較すると、次のようになる。

| 言語 | 多重継承 | カプセル化 | オーバーライド | 抽象クラス | インターフェース |
|:-:|:-:|:-:|:-:|:-:|:-:|
| C++ | ⭕ | ⭕ | virtualのみ | ⭕ | ❌<br />（抽象クラスと純粋仮想関数で対応） |
| Java | ❌ | ⭕ | 任意のメソッド | ⭕ | ⭕ |
| C# | ❌ | ⭕ | virtualのみ | ⭕ | ⭕ |
| JavaScript | ❌ | ⭕<br />（ECMAScript 2022） | 任意のメソッド | ❌ | ❌ |
| Python | ⭕ | ❌<br />（難読化は可能） | 任意のメソッド | ❌<br />（標準ライブラリで対応） | ❌ |

``` python
issubclass(class, classinfo)
```

組み込み関数 `issubclass()` は、第 1 引数に指定したクラスが、第 2 引数に指定したクラスの（直接または間接の）サブクラスであるときに `True` を返し、そうでないときに `False` を返す。第 2 引数にクラスのタプルを指定することもでき、その中のどれか 1 つから継承しているかどうかをチェックする。

Python のクラスはすべて `object` クラスから継承している。

In [ ]:
class MyClass:
    pass

assert issubclass(MyClass, object)

Python におけるオブジェクト指向（object-oriented）は徹底しており、クラス自体もオブジェクトである。クラスもオブジェクトであるため型が存在し、それは `type` となる。つまり、組み込みクラスを含む全てのクラスは `type` のインスタンスであり、これには `object` クラスも含まれる。

In [ ]:
assert isinstance(int, type)
assert isinstance(MyClass, type)
assert isinstance(object, type)

`MyClass` のサブクラスとして新しいクラス `MySubClass` を追加したとき、`MySubClass` は `MyClass` から継承しているが、`type` のインスタンスであることには変わらない。

In [ ]:
class MySubClass(MyClass):
    pass

assert issubclass(MySubClass, MyClass)
assert isinstance(MySubClass, type)

クラスは `type` のインスタンスであるから、`type` そのものはクラスである。`type` クラスの型は `type` である。また、`type` クラスは、他のクラスと同様に `object` クラスから継承している。

In [ ]:
assert isinstance(type, type)
assert issubclass(type, object)

以上のことをまとめると、次の図のようになる（[POSTD の記事](https://postd.cc/pythons-objects-and-classes-a-visual-guide/)から引用）。`m` はクラス `MyClass` のインスタンス、`s` はクラス `MySubClass` のインスタンスとし、型と継承元を示すポインタを描いている。

![](https://postd.cc/wp/wp-content/uploads/2015/11/Python-objects-81.png)

メタクラスとクラス定義
----------------------

`type` のように、クラスの型となるものを**メタクラス**（meta class）という。

`type()` 関数は、3 個の引数が渡される場合、メタクラスのコンストラクタとして振る舞い、新しいクラスオブジェクトを返す。3 個の引数は、左から順に次の属性（およびその元となるデータ）を設定する。

| 特殊属性 | 意味 |
|:--|:----|
| `__name__` | クラス名 |
| `__bases__` | 基底クラスのタプル |
| `__dict__` | クラスの名前空間を管理する辞書（書き換え不可な mappingproxy 型として保持される）。組み込み関数 `vars()` はこの値を返す |

Python ではクラスごとに新たな名前空間が作成される。この名前空間の実体はクラス属性を集めた辞書であり、クラスの `__dict__` 属性としてアクセスできる。

`type()` 関数を使うことでクラスを動的に定義することができる。

次のコードでは、`list` を継承し、要素の文字列を空白区切りで連結するメソッド `concat()` を持つ `StrList` クラス（クラスオブジェクト）を `type()` で生成している。

In [ ]:
def concat(self):
    ret = ""
    for s in self:
        if ret:
            ret += " " + s
        else:
            ret = s
    return ret

StrList = type("StrList", (list,), {"concat": concat})

# インスタンス化
sl = StrList(("peek", "a"))
sl.append("boo")
slang = sl.concat()
print(slang)

peek a boo


クラス定義（class 文）は実行可能な文である。例えば、次のクラス定義:

``` python
class A:
    x = 1
```

このようなクラス定義に対して、Python は内部的に次のようなことを行う:

  1. 新しいローカルスコープ（クラススコープ）を作る。
  2. そのスコープ内でクラス属性 `x` を定義する。
  3. 最後に `type("A", (), {'x': 1})` を呼び出す。

つまり、class 文は「名前空間を作ってから `type()` を呼ぶ糖衣構文」であると言える。

クラス定義は独自のスコープを形成するが、このスコープは属性辞書を構築するためにクラス定義の実行中だけ一時的に使われるものであり、メソッドなどの実行時の名前解決に使われることはない。このため、クラススコープは LEGB ルールの「E（Enclosing）」に含まれていない。実際、メソッドの実行時の名前解決では無視される。

In [ ]:
x = 100
class A:
    x = 200
    y = x + 1  # x は一時的なクラススコープのローカル変数扱い
    def f(self):
        return x  # 実行時はクラスの外側の変数にアクセスする
a = A()
assert a.y == 201
assert a.f() == 100

クラス定義上の内包表記の式は、「ネストされた関数」のように振る舞い、「E（Enclosing）」に含まれないクラススコープを無視する。

In [ ]:
x = 100
class B:
    x = 200
    lst = [x for _ in range(1)]  # この内包表記はグローバル変数にアクセスする
b = B()
assert b.lst == [100]

class 文はデフォルトではメタクラスのコンストラクタとして `type()` 関数を呼び出すが、`metaclass` キーワード引数で別のメタクラスを指定することにより、そのメタクラスのコンストラクタを呼び出すことができる。つまり

``` python
class MyClass(metaclass=Meta):
    pass
```

は

``` python
MyClass = Meta("MyClass", (object,), {})
```

とほとんど等価である。次の例で `MyClass` と `MySubclass` は両方とも `Meta` のインスタンスである:

In [ ]:
class Meta(type):
    pass

class MyClass(metaclass=Meta):
    pass

class MySubclass(MyClass):
    pass

assert isinstance(MyClass, Meta)
assert isinstance(MySubclass, Meta)

クラス定義が実行されると、`metaclass(name, bases, namespace, **kwds)` が呼び出されてクラスオブジェクトが生成されるわけだが、デフォルトのメタクラス `type` か `type` を継承するメタクラスを使っているときは、`type.__new__()` メソッドが呼び出され、オブジェクト生成の主な作業が行われる。

``` python
type.__new__(cls, name, bases, namespace, **kwargs)
```

第 1 引数の `cls` にはメタクラスが渡される。

また、クラスオブジェクトの生成後、処理が戻される前に、`type.__init__()` メソッドが呼び出される。

``` python
type.__init__(self, name, bases, namespace, **kwargs)
```

`type.__new__()` や `type.__init__()` をオーバーライドすれば、クラス生成をカスタマイズできる。

次のコードでは、`type` を継承するメタクラス `Meta` において、`__new__()` をオーバーライドして、追加する属性やメソッドの名前にアンダースコア `'_'` が付くようにしている。また、`__init__()` をオーバーライドして、デフォルトで `say()` メソッドが提供されるようにしている。

In [ ]:
def say(self):
    print("Hello, world!")


class Meta(type):
    def __new__(cls, name, bases, namespace, **kwargs):
        concealment = dict(("_" + k, v) for k, v in namespace.items())
        return super().__new__(cls, name, bases, concealment, **kwargs)

    def __init__(self, name, bases, namespace, **kwargs):
        super().__init__(name, bases, namespace, **kwargs)
        self.say = say


class MyClass(metaclass=Meta):
    def test(self):
        print("This is a test.")


m = MyClass()
m.say()  # say() メソッドはクラス定義では定義されてなく、継承もしていないが、利用可能である
m._test()  # test() メソッドの名前は _test に変更されている
try:
    m.test()  # test という名前でメソッドを呼び出すとエラーが発生する
except AttributeError as err:
    print(err)

Hello, world!
This is a test.
'MyClass' object has no attribute 'test'


super() と MRO
--------------

先のコード例で組み込み関数 `super()` を使用していたが、これは基底クラス（`type`）のメソッドを呼び出しており、メソッドのオーバーライドを行うに際して「基底クラスの機能」＋「アルファ」を実現している。

`super` は組み込みのクラスであり、コンストラクタは次のようになる。

``` python
super()
super(type, object_or_type=None, /)
```

返される `super` オブジェクトは、ドット `.` 構文で使用されると仲介役（proxy）として機能し、後ろに続くメソッドの呼び出しを基底クラスに委譲するという特殊なオブジェクトである。

第 2 引数を指定した場合、そのクラス属性 `__mro__` の値（タプル）の順序に従って基底クラスが検索される。この値は**メソッド解決順序**（method resolution order; **MRO**）と呼ばれる。`__mro__` 属性は `type` と `object` が備えており、全てのクラスで利用できる。

第 1 引数には、検索の起点となるクラスを指定する。第 2 引数により MRO が与えられた場合は、第 1 引数に指定したクラスまではスキップされる。例えば、与えられた MRO が `(D, B, C, A, object)` というもので、第 1 引数に `B` を指定した場合、「`C` → `A` → `object`」という順序でクラスが検索される。

コンストラクタがメソッド中で直接呼び出される場合、どちらの引数も省略できる（ゼロ引数 `super()`）。この場合、第 1 引数は `super()` を使用しているクラス、第 2 引数は `super()` を呼び出しているメソッドの第 1 引数（通常は `self`）となる。

In [ ]:
class A:
    def echo(self):
        print("A")

class B(A):
    pass

class C(B):
    def echo(self):
        s = super()  # s = super(C, self) と同じ
        print(f"{type(s)=}")  # s は super オブジェクト
        s.echo()  # B -> A -> object の順で検索

print(f"object の MRO: {object.__mro__}")
print(f"A の MRO: {A.__mro__}")
print(f"B の MRO: {B.__mro__}")
print(f"C の MRO: {C.__mro__}")
c = C()
super(C, c).echo()  # A.echo() が呼び出される
c.echo()

object の MRO: (<class 'object'>,)
A の MRO: (<class '__main__.A'>, <class 'object'>)
B の MRO: (<class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
C の MRO: (<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
A
type(s)=<class 'super'>
A


上記のコードで `super()` を使用している部分を明示的に `A.echo()` と呼び出しても結果は同じである。しかし、明示的に呼び出す場合、クラス名が変更されるとコードを修正する必要がある。

多重継承
--------

Python では多重継承、すなわち複数の親クラスを持つことができる。多重継承の親クラスをたどっていくと、複数の経路からある 1 つのクラスに行き着くとき、このような継承関係を**ダイヤモンド継承**と呼ぶ。Python の全てのクラスは `object` クラスを基底クラスとしているので、Python の多重継承は常にダイヤモンド継承となる。

ダイヤモンド継承の場合、適切な順序でクラスを検索しないと、同じクラスを 2 回も探そうとする無駄な動作をすることになる。Python は、そのような問題がないように適切な順序で MRO を構築する。具体的には、**C3 線形化**（C3 Linearization）というアルゴリズムに基づいている。

C3 線形化は、各親クラスの MRO をマージするためのアルゴリズムである。まずクラス定義で指定された親クラスの順序（左から右）で、候補とする MRO リストの順番を決める。候補から採用する要素がなければ次の MRO リストを候補とし、要素が採用された場合は最初のリストに戻ることを繰り返す。要素の採用基準は「**候補の先頭が他のリストの先頭以外に現れていない場合に採用**」というもので、採用された要素を全てのリストから取り除く。これにより重複する要素のない一列の順序が得られる。

親クラスが全て `object` クラスのみから継承している場合は、MRO は極めてシンプルである。

例えば、以下のクラス定義の場合:

``` python
class A: ...
class B: ...
class C: ...
class D(A, C, B): ...
```

クラス定義は上から順に実行されるので、最初の 3 つのクラスの MRO が次のように構築される。

  * `A.__mro__` → `(A, object)`
  * `B.__mro__` → `(B, object)`
  * `C.__mro__` → `(C, object)`

Python は MRO を構築するとき、定義するクラス自身を必ず最初の要素とする。このため、`A.__mro__` の値は `(A, object)` となる。

派生クラス `D` の MRO の構築では、`A`、`B`、`C` の各 MRO をマージする。`[A, object]`、`[B, object]`、 `[C, object]` という 3 つのリストを作成する。

  1. 定義 `class D(A, C, B):` の親クラスの順序（左から右）を尊重して、`[A, object]` を最初の候補とする。
  2. `[A, object]` の先頭 `A` が他のリストの先頭以外に含まれていないので採用し取り出す。この時点で 3 つのリストは `[object]`、`[B, object]`、`[C, object]` となっている。候補が最初のリストに戻る（ここでは同じリスト）。
  3. `[object]` の先頭は他のリストの先頭以外に含まれているので不採用。次のリストを候補とする。
  4. `[B, object]` の先頭 `B` が他のリストの先頭以外に含まれていないので採用し取り出す。この時点で 3 つのリストは `[object]`、`[object]`、`[C, object]` となっている。候補が最初のリストに戻る。
  5. 同様のことを繰り返して、`C` が採用され、最終的に `object` だけのリストが残る。

結果として、クラス定義で指定された親クラスの順序どおりに、「`D` → `A` → `C` → `B` → `object`」という MRO が構築される。この順序は直感的に理解できる。

![](https://www.plantuml.com/plantuml/png/SoWkIImgAStDuKhEIImkLiXFoafDBb6mCRM32ZWgs1Z1sCuADiOmjfE2ZGcCxQ9WOsZ71TDQewkhu8Z2SqRXEN4vGXWkNCumXXECuP2Qbm9q3G00)

In [ ]:
class A: ...
class B: ...
class C: ...
class D(A, C, B): ...

print(f"{A.__mro__=}")
print(f"{B.__mro__=}")
print(f"{C.__mro__=}")
print(f"{D.__mro__=}")

A.__mro__=(<class '__main__.A'>, <class 'object'>)
B.__mro__=(<class '__main__.B'>, <class 'object'>)
C.__mro__=(<class '__main__.C'>, <class 'object'>)
D.__mro__=(<class '__main__.D'>, <class '__main__.A'>, <class '__main__.C'>, <class '__main__.B'>, <class 'object'>)


多重継承で親クラスが `object` 以外のクラスから継承していると、途端に MRO は複雑になる。

例えば、以下のクラス定義の場合:

``` python
class A: ...
class B: ...
class C: ...
class D(A): ...
class E(A, B): ...
class F(B): ...
class G(B): ...
class H(D, E, F): ...
```

最初の 7 つのクラスの MRO は次のように構築される。

  * `A.__mro__` → `(A, object)`
  * `B.__mro__` → `(B, object)`
  * `C.__mro__` → `(C, object)`
  * `D.__mro__` → `(D, A, object)`
  * `E.__mro__` → `(E, A, B, object)`
  * `F.__mro__` → `(F, B, object)`
  * `G.__mro__` → `(G, B, object)`

`H` の MRO の構築では、`D`、`E`、`F` の MRO をマージする。`[D, A, object]`、`[E, A, B, object]`、`[F, B, object]` という 3 つのリストを作成する。
  1. 定義の親クラスの順序（左から右）が尊重され、`[D, A, object]` を最初の候補とする。
  2. 先頭 `D` は他のリストの先頭以外に含まれていないので採用して取り出す。この時点で 3 つのリストは `[A, object]`、`[E, A, B, object]`、`[F, B, object]` となっている。候補が最初のリストに戻る（ここでは同じリスト）。
  3. `[A, object]` の先頭は `[E, A, B, object]` の先頭以外に含まれているので不採用。次のリストを候補とする。
  4. `[E, A, B, object]` の先頭 `E` は他のリストの先頭以外に含まれていないので採用して取り出す。この時点で 3 つのリストは `[A, object]`、`[A, B, object]`、`[F, B, object]` となっている。候補が最初のリストに戻る。
  5. `[A, object]` の先頭 `A` は他のリストの先頭以外に含まれていないので採用し、全てのリストから取り出す。この時点で 3 つのリストは `[object]`、`[B, object]`、`[F, B, object]` となっている。候補が最初のリストに戻る（ここでは同じリスト）。
  6. `[object]` の先頭は他のリストの先頭以外に含まれているので不採用。次のリストを候補とする。
  7. `[B, object]` の先頭は `[F, B, object]` の先頭以外に含まれているので不採用。次のリストを候補とする。
  8. `[F, B, object]` の先頭 `F` は他のリストの先頭以外に含まれていないので採用して取り出す。この時点で 3 つのリストは `[object]`、`[B, object]`、`[B, object]` となっている。候補が最初のリストに戻る。
  9. `[object]` の先頭は他のリストの先頭以外に含まれるので不採用。次のリストを候補とする。
  10. `[B, object]` の先頭 `B` は他のリストの先頭以外に含まれていないので採用し、全てのリストから取り出す。最終的に `object` だけのリストが残る。

結果として `H` の MRO は次のように構築される。

  * `H.__mro__` → `(H, D, E, A, F, B, object)`

![](http://www.plantuml.com/plantuml/png/NSqz3eCm50FW_Pt25s3e__A4I4W2rw2KfGgjr8IjX-z0FAIJFtjnb_BqoTiQPOvJIdZVdykSeRUs30PwfYtqIlSb7VJ8pKDFT81UwA7a23sqmejlqy1KpLQj5xF34Lxi7m8nYDinYYS2qIslnxR67m00)

以下のクラス定義を追加した場合:

``` python
class I(F, C): ...
class J(H, I, G): ...
```

C3 線形化によるマージにより、`I` と `J` の MRO は次のように構築される。

  * `I.__mro__` → `(I, F, B, C, object)`
  * `J.__mro__` → `(J, H, D, E, A, I, F, G, B, C, object)`

![](http://www.plantuml.com/plantuml/png/NSv93e9048NXVPtYNO14d4qgXXcQMo2oCOqcThFpy0xjMy3k-sjJBt6-U_ffjDBRpZayBhUXzz0uFlylANH7Pz0JdKFZ5QE0RkWIkgKhw86keKUwWQvf0ztJBVJpc2DUKOHqLjciSaa32gAKR86AgAK8QAGagWMCv4GJq8eXwYz-IeRxTHhj6m00)

`H` を起点としたときの検索順序は維持されていることが注目される。

C3 線形化の特徴は、親クラスの MRO が 子クラスの MRO と矛盾しないことである。この特徴により、基底クラスの検索順序に影響を与えずに派生クラスを追加できる。

In [ ]:
class A: ...
class B: ...
class C: ...
class D(A): ...
class E(A, B): ...
class F(B): ...
class G(B): ...
class H(D, E, F): ...
class I(F, C): ...
class J(H, I, G): ...

print(f"{A.__mro__=}")
print(f"{B.__mro__=}")
print(f"{C.__mro__=}")
print(f"{D.__mro__=}")
print(f"{E.__mro__=}")
print(f"{F.__mro__=}")
print(f"{G.__mro__=}")
print(f"{H.__mro__=}")
print(f"{I.__mro__=}")
print(f"{J.__mro__=}")

A.__mro__=(<class '__main__.A'>, <class 'object'>)
B.__mro__=(<class '__main__.B'>, <class 'object'>)
C.__mro__=(<class '__main__.C'>, <class 'object'>)
D.__mro__=(<class '__main__.D'>, <class '__main__.A'>, <class 'object'>)
E.__mro__=(<class '__main__.E'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)
F.__mro__=(<class '__main__.F'>, <class '__main__.B'>, <class 'object'>)
G.__mro__=(<class '__main__.G'>, <class '__main__.B'>, <class 'object'>)
H.__mro__=(<class '__main__.H'>, <class '__main__.D'>, <class '__main__.E'>, <class '__main__.A'>, <class '__main__.F'>, <class '__main__.B'>, <class 'object'>)
I.__mro__=(<class '__main__.I'>, <class '__main__.F'>, <class '__main__.B'>, <class '__main__.C'>, <class 'object'>)
J.__mro__=(<class '__main__.J'>, <class '__main__.H'>, <class '__main__.D'>, <class '__main__.E'>, <class '__main__.A'>, <class '__main__.I'>, <class '__main__.F'>, <class '__main__.G'>, <class '__main__.B'>, <class '__main__.C'>, <class 'object'>)


親クラスが `object` 以外のクラスから継承しているような多重継承は、MRO の理解・追跡が難しくなるので、あまり使うべきではないであろう。

特殊メソッド
------------

`type` の基底クラスである `object` クラスも、`__new__()` や `__init__()` メソッドを持つ。これらのように、オブジェクトの振る舞いを決める、特殊な名前のメソッドが定義されている。

### コンストラクタとイニシャライザ

| 特殊メソッド | 機能 | 備考 |
|:--|:----|:----|
| `object.__new__(cls[, ...])` | クラスの新しいインスタンスを生成して返す。第 1 引数 `cls` は生成対象のクラス。<br /><br />残りの引数はクラス呼び出し（`Class(...)`）で渡されたものがそのまま渡される | クラス呼び出し式が評価されるときに最<br /><br />初に呼び出される |
| `object.__init__(self[, ...])` | インスタンスの初期設定を行う。`self` は新たに生成されたインスタンスで、残り<br /><br />の引数はクラス呼び出しで渡されたものである | インスタンスが `__new__()` によって生成<br /><br />された後、それが呼び出し元に返される前<br /><br />に呼び出される |

一般に、**コンストラクタ**（constructor） とは、オブジェクト生成時に自動的に呼び出されるメソッドを指す。C++ や Java などのクラスベース言語では、コンストラクタがインスタンスの初期化を担う。

一方、Python では、インスタンスを生成して返すメソッド `__new__()` と、インスタンスを初期化するメソッド `__init__()` が明確に分かれている。そのため、`__new__()` を「コンストラクタ」、`__init__()` を「**イニシャライザ**（initializer）」と呼ぶことがある。ただし、Python の一般的な文脈では「コンストラクタ」という語が曖昧に使われることも多い。例えば、Java の感覚で `__init__()` をコンストラクタと呼ぶことがあり、また、JavaScript の感覚でクラス呼び出しそのもの（`Class()`）をコンストラクタ呼び出しとみなして「コンストラクタ」と呼ぶことがある。

`__new__()` と `__init__()` は連携してオブジェクトを構成する。つまり、`__new__()` がオブジェクトを作成し、`__init__()` がそれをカスタマイズする。このため、次の点に注意する。

  * `__new__()` が `cls` のインスタンスを返さない場合、インスタンスの `__init__()` メソッドは呼び出されない。
  * `__init__()` は初期化専用であり、`None` 以外を返すと `TypeError` が送出される。

基底クラスとその派生クラスがともに `__init__()` メソッドを持つ場合、派生クラスの `__init__()` メソッドは、`super().__init__([args...])` のように基底クラスの `__init__()` メソッドを明示的に呼び出して、インスタンスの基底クラス部分が適切に初期化されること保証しなければならない。Python では Java と異なり、基底クラスの `__init__()` が自動で呼ばれることはない。

### Singleton パターン

クラス設計における定石集を**デザインパターン**（design pattern）と呼ぶ。GoF（Gang of Four; 4 人組）と呼ばれる 4 人の著者による書籍、『オブジェクト指向における再利用のためのデザインパターン』の中で取り上げられた 23 種類のデザインパターンが有名である。ただし、これらのデザインパターンは Python 向けのものではないため、Python ではあまり使われないものや、Python の言語仕様として組み込まれているものがある。

**Singleton パターン**は、デザインパターンの 1 つで、そのクラスのインスタンスが 1 つしか生成されないことを保証する設計パターンである。そのようにインスタンス化が制限されるクラスのことを**シングルトンクラス**と呼び、シングルトンクラスの唯一のインスタンスを**シングルトン**と呼ぶ。

Python では、`None`、`NotImplemented`、`Ellipsis` が組み込みのシングルトンであるが、これらは値を変更できない特殊なシングルトンである。

In [ ]:
assert type(None)() is None
assert type(NotImplemented)() is NotImplemented
assert type(Ellipsis)() is Ellipsis

Singleton パターンの一般的なクラス図は、次のようなものとなる。

![](http://www.plantuml.com/plantuml/png/JOwn2i9044Jx_Oej4oHWVOL4h5Bd2oxd4bOlUt8tQHJ_tQ32fTfbviqmVO7FM9T8vQxsyDal7A8lvGnadLR849AD-XG-qjVWKMsEWcJyeexLjWOBugfENVsRbWwpOFXJJRldtfIccmJm9QeOw7UuErN6kLsQUh7hjlK3)

シングルトンのクラスは、同じ型のインスタンスを格納するクラス変数を持つ。この変数と、シングルトンのコンストラクタはクライアント・コードから隠れており、`get­Instance()` メソッドの呼び出しがシングルトンのオブジェクトを取得する唯一の方法となっている。`get­Instance()` メソッドの中では、クラス変数にインスタンスが格納されていなければインスタンスを生成して格納する。以後、`get­Instance()` はクラス変数が保持するインスタンスのみを返す。このロジックにより、生成されるインスタンスは 1 つだけであることが保証される。

Python では、コンストラクタを隠すということができないので、`object.__new__()` メソッドをオーバーライドして、これに `get­Instance()` の機能を持たせることにする。

次のコードは、ユーザー定義のシングルトンの例である。

In [ ]:
class Singleton:
    def __new__(cls, *args, **kwargs):
        if not hasattr(cls, "_instance"):
            cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self, input=0):
        self.input = input


if __name__ == "__main__":
    A = Singleton(1)
    B = Singleton(2)
    assert A is B
    assert A.input == B.input == 2

`__new__()` の中では、自身のインスタンスを保持する内部クラス変数 `_instance` がなければインスタンスを格納する。以後、`Singleton` のコンストラクタは `_instance` が保持するインスタンスのみを返す。インスタンス作成のために使う `super().__new__(cls)` メソッドは、今は基底クラスを定義していないので `object.__new__(cls)` と同一である。残りの可変長引数 `args` と `kwargs` はクラスのコンストラクタ式に渡され、さらに `__init__()` を呼び出す際にその引数に渡される。

シングルトンクラスのインスタンスはアプリケーション全体で共有するので、Singleton パターンはアプリケーションのグローバルな状態を管理するのに適する。このことは利点である反面、テストを困難にするという欠点にもなる。プログラムの各クラスに対して独立に行うべきテストまでがシングルトンの状態を共有することになるため、前のテストで変更したシングルトンの状態が、次のテストで引き継がれるということが起こってしまう。

### &#x5f;&#x5f;repr&#x5f;&#x5f;() と &#x5f;&#x5f;str&#x5f;&#x5f;()

| 特殊メソッド | 機能 | 備考 |
|:--|:----|:----|
| `object.__repr__(self)` | 同じ値のオブジェクトを再生成するのに使える、有効な Python 式のような文字列<br /><br />を返す | 組み込み関数 `repr()` によって呼び出される |
| `object.__str__(self)` | オブジェクトを表現する文字列を返す（`object` のデフォルト実装は `__repr__()` <br /><br />を呼び出す） | 組み込み関数 `str()`、`format()`、`print()` <br /><br />によって呼び出される |

`object.__repr__()` の実装は、 `'<ClassName object at 0x...>'` の形式で文字列を返す。この `0x...` 部分は通常 CPython 実装におけるメモリアドレスである。

In [ ]:
x = object()

class A:
    def __init__(self, name):
        self.name = name

a = A("hoge")
repr(x), repr(a)

('<object object at 0x790915030d50>', '<__main__.A object at 0x79091480f9e0>')

[公式ドキュメント](https://docs.python.org/ja/3/reference/datamodel.html#object.__repr__)によれば、`__repr__()` は「可能なら同じ値のオブジェクトを再生成できる有効な Python 式のような文字列を返すべき」とされている。そのため、クラスのインスタンスをわかりやすく表現したい場合は `__repr__()` を定義するのが望ましい。

なお `__str__()` は、有効な Python 式の表現を返すことが期待されないという点で `__repr__()` とは異なるので、より便利な、または簡潔な表現を使用することができる。

In [ ]:
class A:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"A('{self.name}')"

a = A("hoge")
repr(a)

"A('hoge')"

### 演算子オーバーロード

| 特殊メソッド | 機能 | 備考 |
|:--|:----|:----|
| `object.__eq__(self, other)` | 等しい | 演算子 `==` によって呼び出される |
| `object.__ne__(self, other)` | 等しくない | 演算子 `!=` によって呼び出される |
| `object.__lt__(self, other)` | 小なり | 演算子 `<` によって呼び出される |
| `object.__le__(self, other)` | 小なりイコール | 演算子 `<=` によって呼び出される |
| `object.__gt__(self, other)` | 大なり | 演算子 `>` によって呼び出される |
| `object.__ge__(self, other)` | 大なりイコール | 演算子 `>=` によって呼び出される |

式 `x == y` では、まず `x.__eq__(y)` の呼び出しが試みられ、`__eq__()` が実装されていない場合、または、`__eq__()` が `NotImplemented` を返す場合、`y.__eq__(x)` の呼び出しが試みられる。他の `other` を引数に持つ特殊メソッドでも同様の動作となる。

In [ ]:
class X:
    def __init__(self, val):
        self.val = val

class Y:
    def __init__(self, val):
        self.val = val

    def __eq__(self, other):
        return other.val != self.val  # あえて反対の評価を返す

assert X(0) == Y(1)  # Y(1).__eq__(X(0)) が呼び出される

`==` や `!=` 演算子によって呼び出される特殊メソッドがクラスで独自に定義される可能性があることから、PEP 8 は、`None` に対しては `==` や `!=` でなく、`is` や `is not` を使うことを推奨している。

In [ ]:
a = None
assert a is None  # PEP 8 推奨
assert a == None  # PEP 8 非推奨

特定の型に属するオブジェクトが持つような特殊メソッドもある:

| メソッド | 機能 | 備考 |
|:--|:----|:----|
| `__mul__(self, other)` | 乗算 | 演算子 `*` によって呼び出される |
| `__matmul__(self, other)` | 行列積 | 演算子 `@` によって呼び出される |
| `__truediv__(self, other)` | 除算 | 演算子 `/` によって呼び出される |
| `__floordiv__(self, other)` | 整数除法 | 演算子 `//` によって呼び出される |
| `__mod__(self, other)` | 剰余 | 演算子 `%` によって呼び出される |
| `__add__(self, other)` | 加算 | 演算子 `+` によって呼び出される |
| `__sub__(self, other)` | 減算 | 演算子 `-` によって呼び出される |
| `__getitem__(self, key)` | インデックス参照 | 演算子 `[]` によって呼び出される |
| `__setitem__(self, key, value)` | インデックス参照 | 演算子 `[]` によって呼び出される |
| `__delitem__(self, key)` | インデックス参照 | 演算子 `[]` によって呼び出される |

式 `x * y` の評価では、まず `x.__mul__(y)` の呼び出しが試みられ、`__mul__()` が実装されていない場合、または、`__mul__()` が `NotImplemented` を返す場合、`y.__mul__(x)` の呼び出しが試みられる。他の `other` を引数に持つ特殊メソッドでも同様の動作となる。

式 `x[i]` の評価では `x.__getitem__(i)` の呼び出しが試みられ、 式 `x[i] = v` の評価では `x.__setitem__(i, v)` の呼び出しが試みられる。`x[i]` の削除では `__delitem__(i)` の呼び出しが試みられる。

これらの特殊メソッドを独自に定義することによって、オブジェクトの振る舞いをカスタマイズすることができる。これは、Python で演算子オーバーロードを実現する方法である。なお、`__matmul__()` をサポートする組み込み型や標準ライブラリーはなく、行列積演算子 `@` は初めからサードパーティーのライブラリーで実装されることが期待されている（実際、`Numpy` で実装された）。

In [ ]:
class MyClass:
    def __init__(self, num):
        self.val = num
        self.even_num = 0
        self.odd_num = 1

    def __add__(self, other):
        if isinstance(other, MyClass):
            num = int(str(self.val) + str(other.val))
            return MyClass(num)
        else:
            return NotImplemented

    def __getitem__(self, key):
        if not isinstance(key, int):
            raise TypeError("key が整数でない")
        if key % 2 == 0:
            return self.even_num
        else:
            return self.odd_num

    def __setitem__(self, key, value):
        if not isinstance(key, int):
            raise TypeError("key が整数でない")
        if key % 2 == 0:
            self.even_num = value
        else:
            self.odd_num = value


m1 = MyClass(10)
m2 = MyClass(24)
assert (m1 + m2).val == 1024
m1[31] = 3
assert m1[1] == 3

### スライスオブジェクト

シーケンスのインデックス参照は、内部的に添字表記から組み込みの `slice` オブジェクトを作成して `__getitem__()` メソッドに渡すという処理をしている。添字表記の中に直接 `slice` オブジェクトを指定することもできる。

`slice` のコンストラクタは、引数の数が異なるもので 2 つ存在する:

``` python
slice(stop)
slice(start, stop, step=None)
```

これらは、`range(start, stop, step)` で指定されたインデックスのセットを表す `slice` オブジェクトを返す。

`slice` オブジェクトは、読み出し専用の `start`, `stop`, `step` 属性を持つ。これらは単にコンストラクタ引数の値（またはそのデフォルト値）を返す。`start` と `step` のデフォルト値は `None`。

添字表記に対応する `slice` オブジェクトの属性は、次のとおり。

| 添字表記 | `start` | `stop` | `step` |
|:---|:---|:---|:---|
| `[i:j]` | `i` | `j` | `None` |
| `[i:j:k]` | `i` | `j` | `k` |
| `[i:]` | `i` | `None` | `None` |
| `[:j]` |`None` | `j` | `None` |
| `[::k]` |`None` | `None` | `k` |

In [ ]:
a = [x for x in range(10)]
assert a[2:8] == a[slice(2, 8)] == [2, 3, 4, 5, 6, 7]
assert a[::-1] == a[slice(None, None, -1)] == [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

### ハッシュ可能

| 特殊メソッド | 機能 | 備考 |
|:--|:----|:----|
| `object.__hash__(self)` | ハッシュ値を返す | 組み込み関数 `hash()` によって呼び出される |

オブジェクトが**ハッシュ可能**（hashable）であるとは、オブジェクトが特殊メソッド `__eq__()` および `__hash__()` を持ち、かつ、`__hash__()` が次の条件をみたしていることをいう。

  1. 生存期間中変わらない整数（ハッシュ値）を返す。
  2. 値が等しいハッシュ可能オブジェクト同士では、同じハッシュ値を返す。

ハッシュ可能なオブジェクトに対して組み込み関数 `hash()` を実行すると、その `__hash__()` メソッドが（存在すれば）呼び出されハッシュ値を返す。

ハッシュ可能なオブジェクトは `dict` オブジェクト（辞書）のキーや、`set` オブジェクト（集合）および `frozenset` オブジェクト（集合）の要素として使える。辞書や集合のデータ構造は内部でハッシュ値を使っているからである。なお、`set` オブジェクトはミュータブルだが、`frozenset` オブジェクトはイミュータブルである。

オブジェクトがイミュータブルであることと、ハッシュ可能であることは異なる概念であるが、Python のイミュータブルな組み込みオブジェクトは、ほとんどがハッシュ可能である。この例外は `tuple` と `frozenset` の 2 つであり、どちらもイミュータブルであるが、すべての要素がハッシュ可能であるときのみハッシュ可能となる。ミュータブルな組み込みオブジェクトはすべてハッシュ不可能である。

`object` 型のインスタンスはハッシュ可能である。`object` 型のインスタンス `x` において、`x.__hash__()` は `x == y` が `x is y` と `hash(x) == hash(y)` の両方を意味するような適切な値を返す（`id()` の戻り値から計算している）。ユーザー定義クラスは、`__eq__()` と `__hash__()` メソッドを `object` クラスから継承する。つまり、ユーザー定義クラスでは基本的にインスタンスがハッシュ可能である。しかし、クラス定義に `__hash__ = None` とするオーバーライドを含めると、ハッシュ不能となる。また、`__eq__()` をオーバーライドしていて `__hash__()` を定義していないクラスでは、`__hash__()` は暗黙的に `None` に設定され、ハッシュ不能となる。

In [ ]:
class C1:
    def __init__(self, val):
        self.val = val

    def __eq__(self, other):
        if isinstance(other, C1):
            return self.val == other.val
        else:
            return NotImplemented


class C2:
    def __init__(self, val):
        self.val = val

    def __eq__(self, other):
        if isinstance(other, C2):
            return self.val == other.val
        else:
            return NotImplemented

    def __hash__(self):
        return super().__hash__()


c1 = C1(1)
assert c1.__hash__ is None  # __hash__ 属性が自動的に None で定義される
c2 = C2(2)
d2 = {c2: 2}  # c2 はハッシュ可能なので辞書のキーに使用できる

### 真理値判定

いくつかのプログラミング言語（例えば JavaScript）は、広範囲に暗黙の型変換（キャスト）を行う。これに対して Python は、Zen of Python の一節「Explicit is better than implicit.」（明示的であることは、暗黙的であることよりも良い。）という設計哲学に基づき、暗黙の型変換を極めて限定的にしか認めていない。

JavaScript では if 文や while 文などの条件式において任意の値が自動的に boolean 型へ変換されるが、Python はそのような単純な変換を行わない。代わりに、「評価ルール」として式を真理値に解釈する仕組みを採用している。

この「評価ルール」は、真理値を解釈する場面を定める**真理値コンテキスト**（truth value context）と、真理値をどのように判定するかを定める**真理値判定のプロトコル**から構成される。

代表的な真理値コンテキストは以下の通り。

  * **if 文の条件**: 式を記述できる（セイウチ演算子を持つ代入式も可）。
  * **while 文の条件**: 式を記述できる（セイウチ演算子を持つ代入式も可）。
  * **条件演算（if…else）の被演算子（オペランド）**: ブール式を記述できる。
  * **ブール演算の被演算子（オペランド）**: ブール式を記述できる。
  * **組み込み関数 bool() の引数**: ブール式を記述できる。

どの真理値コンテキストにも、アトムとしてオブジェクトを記述できる。Python は、真理値コンテキストにおいてオブジェクトを評価するとき、次のプロトコルに従う：

  1. オブジェクトに `__bool__()` メソッドがあれば、それを呼び出す（戻り値は `True` か `False` のどちらかでなければならない）
  2. `__bool__()` が定義されていなければ、`__len__()` メソッドを呼び出し、返された長さが 0 なら `False`、それ以外は `True`
  3. どちらも定義されていなければ、常に `True`

真理値判定のプロトコルにより、オブジェクトは動的に真理値を判定される。

次のオブジェクトは真理値コンテキストにおいて全て `False` と判定され、値としてこれらを返す式は `False` と判定される。

``` python
None, False, 0, 0.0, 0j, '', (), [], {}, set(), range(0)
```

組み込みの None 型、`bool` 型、`int` 型、`float` 型、`complex` 型のオブジェクトは、全て `__bool__()` メソッドを持つ。これらのオブジェクトのうち、`None`、`False`、`0`、`0.0`、`0j` のみ `__bool__()` メソッドが `False` を返す。

シーケンス（文字列、リスト、タプルなど）は `__len__()` メソッドを持つオブジェクトである。`__len__()` はシーケンスの長さを返し、組み込み関数 `len()` によって呼び出されるほか、真理値コンテキストにおいて真理値判定のためにも呼び出される。

In [ ]:
None.__bool__(), False.__bool__(), (0).__bool__(), (0.0).__bool__(), (0j).__bool__()

(False, False, False, False, False)

In [ ]:
''.__len__(), ().__len__(), [].__len__(), {}.__len__(), set().__len__(), range(0).__len__()

(0, 0, 0, 0, 0, 0)

`'0'` と `' '`（空白文字）は、文字列として長さが 0 ではないから真理値判定は `True` となる。

In [ ]:
bool('0'), bool(' ')

(True, True)

以下は、PEP 8 による推奨事項。

シーケンスが空であるかどうかを調べる場合、真理値コンテキストにおいて空のシーケンスが `False` に判定されることを利用できる。この場合に `len()` を実行して長さを調べるのは、長さを二重に評価することになって無駄である。

``` python
# 正しい:
if not seq:
if seq:

# 間違い:
if len(seq):
if not len(seq):
```

`bool` 型の値と `True` や `False` を比較するのに `==` や `is` を使うべきではない。`bool` 型オブジェクト自体を式として使えば十分なのであって、`==` 演算子や `is` 演算子を加えるのは、無意味な真理値判定を加えることになって無駄である。

``` python
# 正しい:
if is_file:

# 間違い:
if is_file == True:
if is_file is True:
```

### &#x5f;&#x5f;copy&#x5f;&#x5f;()・&#x5f;&#x5f;deepcopy&#x5f;&#x5f;()・&#x5f;&#x5f;replace&#x5f;&#x5f;()

`__copy__()` メソッドや `__deepcopy__()` メソッドの定義により、それぞれ `copy.copy()` 関数と `copy.deepcopy()` 関数の振る舞いを制御できる。

``` python
__copy__(self)
```

`copy.copy()` 関数によって呼び出される。

In [14]:
import copy

class MyClass:
    def __init__(self, x):
        self.x = x

    def __copy__(self):
        return MyClass(self.x)

    def __eq__(self, other):
        return other.x == self.x

obj1 = MyClass(10)
obj2 = copy.copy(obj1)  # copy.copy を使ってコピー（__copy__が呼ばれる）
assert obj1 == obj2
obj2.x = 20
assert obj1 != obj2  # 変更が obj1 に及ばない

``` python
__deepcopy__(self, memo)
```

`copy.deepcopy()` 関数によって呼び出される。`memo` 引数はメモ用の辞書であり、`copy.deepcopy()` 関数が内部で使用するものでユーザーが用意したり操作するものではない。このメソッドの実装で構成要素の深いコピーが必要な場合は、その要素を第 1 引数、`memo` 辞書を第 2 引数として `copy.deepcopy()`関数を呼び出す必要がある。

In [12]:
import copy

class MyClass:
    def __init__(self, x):
        self.x = x

    def __deepcopy__(self, memo):
        return MyClass(copy.deepcopy(self.x, memo))

    def __eq__(self, other):
        return other.x == self.x

vals = ["a", "b", ["c", "d"]]  # ネストされたリスト
obj1 = MyClass(vals)
obj2 = copy.deepcopy(obj1)
assert obj1 == obj2
obj2.x[2][0] = "f"
assert obj1 != obj2  # 変更が obj1 に及ばない

Python 3.13 からは、`copy` モジュールに次の関数が追加された。

``` python
copy.replace(obj, /, **changes)
```

この関数は、第 1 引数 `obj` が持つ `__replace__()` メソッドを呼び出し、その結果を返す。第 2 引数以降のキーワード引数は `__replace__()` メソッドに渡される。

`__replace__()` は、オブジェクトの属性（の一部）を書き換えて、同じ型の新しいオブジェクトを作成するメソッドである。`replace()` メソッドによって呼び出される。

任意のオブジェクトに `__replace__()` が定義されていれば、`replace()` メソッドなくても `copy.replace()` が使える。

``` python
import copy

class MyClass:
    def __init__(self, x):
        self.x = x

    def __replace__(self, **changes):
        return MyClass(copy.deepcopy(changes.get("x", self.x)))

obj1 = MyClass(10)
obj2 = copy.replace(obj1, x=20)
assert obj1.x == 10 and obj2.x == 20
```